In [1]:
LOCAL = False

import numpy as np
import os, sys
import pandas as pd

#import matplotlib.pyplot as plt
#import seaborn as sns
#from scipy import stats


import warnings
warnings.filterwarnings("ignore")


if LOCAL is True:
    sys.path.append("/Users/sarahfong/tools/py_")
    PATH = "/Users/sarahfong/Desktop/local_data/Biomarin_Jun_2023/"
    RE = os.path.join(PATH, "results")
    CONFIG_NAME="config.local.neuron.mpra.ini"
    RE_MAPS = os.path.join(RE, "enh_maps")
else:
    PATH="/wynton/group/ahituv/biomarin/data"
    RE = "/wynton/group/ahituv/fongsl/projects/biomarin/results"
    RE_MAPS = os.path.join(RE, "enh_maps")
    CONFIG_NAME="config.neuron.mpra.orig.ini"

# nomination dataframe
NOMS = os.path.join(PATH, "GABA_GLUT_DF.for.noms.tsv")

import config_readwrite as crw
import plot_params as pp
pp.fonts()

('sans-serif', 'Arial', 18)

In [2]:
config, cfn = crw.read(CONFIG_NAME)

section = 'lib2'
crw.check(config, section)

In [3]:
RE = config["path"]["results_path"]
DELTA_ACTIVITY = config[section]["delta.mpra_centered"]
DELTA_ACTIVITY_Z = config[section]["delta.mpra_centeredz"]

FASTA = config[section]["fasta"]

TILE_BED = config[section]["tile_bed"]
ENH_BED = config[section]["enh_bed"]

GREAT_GENE_TSV = config[section]["great_nearest"]
META_DATA = config[section]["metadata"]

SIG_ACTIVITY_Z = config[section]["sig-wilcoxon"]
VISTA = config[section]["vista"]

SUPPORT = config[section]["support"] # neighboring tile support
TFBS = config["tfbs2"]["vierstra.motif.loc.nored"] # TFBS archetypes where score is in 95% of motif scores, binned by length, no redundants
# i.e. the strongest of the motif matches. 

LOSSES_AND_GAINS = config["tfbs2"]["vierstra.motif.loc.nored.loss.gain"]  # write the tiles with losses and gains only. 

# BEDS to write

In [24]:
TILE_BED_TRACK = os.path.join(RE, "track.tile.bed")
ENH_BED_TRACK = os.path.join(RE, "track.enh.bed")
TFBS_BED_TRACK = os.path.join(RE, "track.tfbs.bed")
GABA_BED_TRACK = os.path.join(RE, "track.gaba.MPRA.bed")
GLUT_BED_TRACK = os.path.join(RE, "track.glut.MPRA.bed")
#TFBS_BED_TRACK = os.path.join(RE, "track.tfbs.bed")

In [5]:
tile = pd.read_csv(ENH_BED, sep='\t').dropna()

# change datatype 
tile[["start.tile", "end.tile", "start.enh", "end.enh"]]=tile[["start.tile", "end.tile", "start.enh", "end.enh"]].astype(int)

In [14]:
tile_track_cols = ["#chr", "start.tile", "end.tile", "name"]
enh_track_cols = ["#chr", "start.enh", "end.enh", "enh.name"]

In [15]:
tile[tile_track_cols].drop_duplicates().to_csv(TILE_BED_TRACK, sep='\t', index=False)
tile[enh_track_cols].drop_duplicates().to_csv(ENH_BED_TRACK, sep='\t', index=False)

## TFBS

In [8]:
tf = pd.read_csv(TFBS, sep='\t').drop_duplicates()
tf.head()

,#chr,start_tile,end_tile,#chr_arch,start_arch,end_arch,group,arch,strand,model,num_models,overlap,motif_len,motif_overlap,start_motif,end_motif,thresh_score,motif_len_thesh,name
0,chr1,10117,10387,chr1,10386,10400,NR/3,10.7457,-,COT1_HUMAN.H11MO.0.C,3,1,14,False,269,283,True,10.7289,chr1:10117-10387
1,chr1,10133,10403,chr1,10386,10400,NR/3,10.7457,-,COT1_HUMAN.H11MO.0.C,3,14,14,True,253,267,True,10.7289,chr1:10133-10403
2,chr1,10137,10407,chr1,10386,10400,NR/3,10.7457,-,COT1_HUMAN.H11MO.0.C,3,14,14,True,249,263,True,10.7289,chr1:10137-10407
3,chr1,10153,10423,chr1,10386,10400,NR/3,10.7457,-,COT1_HUMAN.H11MO.0.C,3,14,14,True,233,247,True,10.7289,chr1:10153-10423
4,chr1,10157,10427,chr1,10386,10400,NR/3,10.7457,-,COT1_HUMAN.H11MO.0.C,3,14,14,True,229,243,True,10.7289,chr1:10157-10427


In [9]:
tf_track_cols = ["#chr", "start_arch", "end_arch", "group", "arch", "strand", "start_arch", "end_arch", ]
tf[tf_track_cols].drop_duplicates().to_csv(TFBS_BED_TRACK, sep='\t', index=False)

In [10]:
tf[tf_track_cols].drop_duplicates()

,#chr,start_arch,end_arch,group,arch,strand,start_arch,end_arch
0,chr1,10386,10400,NR/3,10.7457,-,10386,10400
10,chr1,10480,10489,MBD2,11.1198,-,10480,10489
15,chr1,10502,10518,NR/16,11.5888,-,10502,10518
25,chr1,10559,10575,HEN1,10.8754,-,10559,10575
26,chr1,1692083,1692101,ZNF554,10.5594,-,1692083,1692101
...,...,...,...,...,...,...,...,...
244388,chrY,19744196,19744209,TEAD,10.8798,-,19744196,19744209
244394,chrY,19744460,19744473,NFI/3,11.1223,-,19744460,19744473
244408,chrY,19744696,19744707,YY1,11.6812,-,19744696,19744707
244411,chrY,19744700,19744713,NR/11,11.9466,-,19744700,19744713


# active

In [11]:
active = pd.read_csv(DELTA_ACTIVITY_Z, sep='\t')
active.head()

,name,gaba-label,glut-label,sig,celltype_dif,gaba,glut,delta.gaba-glut,fdr,-log10p_fdr,bs
0,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,False,False,True,-2.699699,-1.696544,-1.003154,0.119390,0.923032,False
1,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,False,False,True,-2.683608,-1.740263,-0.943346,0.493535,0.306682,False
2,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",False,False,False,False,-2.228061,-1.842682,-0.385379,0.728743,0.137425,False
3,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,False,False,True,-2.666574,-1.554678,-1.111896,0.364764,0.437989,False
4,"Backgroundseq172hr_top_98|Pos1:115,Pos2:155|Mo...",silencing,silencing,False,False,-2.756752,-2.505154,-0.251599,0.801537,0.096076,False


In [17]:
active = pd.merge(tile[tile_track_cols], active)

In [22]:
active_gaba_cols = tile_track_cols[::][:-1] + ["gaba"]
active_glut_cols = tile_track_cols[::][:-1] + ["glut"]

In [34]:
active[active_gaba_cols].drop_duplicates().to_csv(GABA_BED_TRACK, sep='\t', index=False, header=False)
active[active_glut_cols].drop_duplicates().to_csv(GLUT_BED_TRACK, sep='\t', index=False, header=False)

In [26]:
GABA_BED_TRACK

'/wynton/group/ahituv/fongsl/projects/biomarin/results/track.gaba.MPRA.bed'

In [33]:
active[active_gaba_cols].drop_duplicates().sort_values(by=active_gaba_cols[:2])#.drop_duplicates()

,#chr,start.tile,end.tile,gaba
0,chr1,1692002,1692272,0.704883
1,chr1,1692022,1692292,0.815943
2,chr1,1692042,1692312,0.714686
3,chr1,1692062,1692332,0.256859
4,chr1,1692082,1692352,0.326321
...,...,...,...,...
34044,chrY,19745320,19745590,1.180058
34045,chrY,19745340,19745610,1.002419
34046,chrY,19745360,19745630,1.140756
34047,chrY,19745380,19745650,0.818851


In [35]:
active[active_gaba_cols].dtypes

#chr           object
start.tile      int64
end.tile        int64
gaba          float64
dtype: object